In [1]:
import ipywidgets as widgets
from IPython.display import display

# GUI elements

api_header = widgets.HBox([widgets.Label(value="Enter Genius API Key")])

api_text = widgets.Text(
    value='',
    placeholder='Enter Genius API Key..',
    description='API:',
    disabled=False
)

api_button = widgets.Button(
    description='Store API',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Store API Key',
    icon='save' # (FontAwesome names without the `fa-` prefix)
)

header = widgets.HBox([widgets.Label(value="Sound Cloud Search")])

song_text = widgets.Text(
    value='',
    placeholder='Enter song name..',
    description='Song:',
    disabled=False
)

artist_text = widgets.Text(
    value='',
    placeholder='Enter artist name..',
    description='Artist:',
    disabled=False
)

generate_button = widgets.Button(
    description='Generate Cloud',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Generate Word Cloud',
    icon='cloud' # (FontAwesome names without the `fa-` prefix)
)
profanity_check = widgets.Checkbox(
            value = False,
            description = 'Remove Profanity',
            disabled = False,
            indent = True)

clear_button = widgets.Button(
    description='Clear Search',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear Search',
    icon='times-circle' # (FontAwesome names without the `fa-` prefix)
)

output = widgets.Output()

In [2]:
# lyricsgenius

import lyricsgenius

genius = lyricsgenius.Genius(api_text.value, remove_section_headers = True)

In [3]:
# Button handling
from collections import Counter
from itertools import dropwhile
from better_profanity import profanity

lyric_count = Counter()
profanity.load_censor_words() #loads the list of bad words into the library

basic_words=["the", "i", "a", "be", "to", "of", "and", "in", "it", "for", "not", "on", "with", "he", "as", "you", "do", "this", "but", "his"]
#20 most common words in englsih words may switch to 10 for the sake of songs
def api_button_clicked(b):
    genius = lyricsgenius.Genius(api_text.value, remove_section_headers = True)
    with output:
        print("[DEBUG - api_button_clicked]\nAPI key:", api_text.value, "\n")

def generate_button_clicked(b):
    with output:
        print("[DEBUG - generate_button_clicked]\nSong:", song_text.value, "\nArtist:", artist_text.value, "\n")
        try:
            song = genius.search_song(song_text.value, artist_text.value) #text_format = ""
            lyrics = song.lyrics.replace('-', ' ').split()
            lyrics = [x.lower() for x in lyrics] # converts all values in list to lowercase
            lyrics = [x.replace(',','') for x in lyrics] # removes all commas 
            lyrics = [x.replace('(', '') for x in lyrics] #removes parenthesis
            lyrics = [x.replace(')', '') for x in lyrics]
            
            for x in list(lyrics): #remove basic words such as "the" 
                if x in basic_words:
                    lyrics.remove(x)
            if profanity_check.value == True: #If profanity box was clicked remove profanity
                for x in list(lyrics):
                    if profanity.contains_profanity(x):
                        lyrics.remove(x)
            #censoredLyrics = profanity.censor(lyrics)
            #print(censoredLyrics)
            #lyrics = [x.replace('-',' ') for x in lyrics] # removes all dashes have to seperate the words
            #for x in censoredLyrics:
                #print(x)
            
            global lyric_count
            lyric_count = Counter(lyrics)

            # Remove lyrics where frequency is >= 4, needs to be dynamic based on song
            for key, count in dropwhile(lambda key_count: key_count[1] >= 3, lyric_count.most_common()):
                del lyric_count[key]
                
            print("[DEBUG - getting lyrics]\n",lyric_count,"\n")
        except TypeError as err:
            print("\n\t--- ERROR - Enter API key correctly ---\t\n")
        
def clear_button_clicked(b):
    song_text.value = ''
    artist_text.value = ''
    with output:
        print("[DEBUG - clear_clicked()]\nCleared 'song_text' and 'artist_text'\n")
                
api_button.on_click(api_button_clicked)
generate_button.on_click(generate_button_clicked)
clear_button.on_click(clear_button_clicked)

In [4]:
# Display GUI
display(api_header, api_text, api_button, header, song_text, artist_text, profanity_check, generate_button, clear_button, output)

Text(value='', description='API:', placeholder='Enter Genius API Key..')

Button(button_style='info', description='Store API', icon='save', style=ButtonStyle(), tooltip='Store API Key'…

Text(value='', description='Song:', placeholder='Enter song name..')

Text(value='', description='Artist:', placeholder='Enter artist name..')

Checkbox(value=False, description='Remove Profanity')

Button(button_style='success', description='Generate Cloud', icon='cloud', style=ButtonStyle(), tooltip='Gener…

Button(button_style='danger', description='Clear Search', icon='times-circle', style=ButtonStyle(), tooltip='C…

Output()

In [7]:
# Canvas

from ipycanvas import Canvas
import random

# creating canvas
canvas = Canvas(width=800, height=800)

# setting font size
canvas.font = '32px serif'

# stroke_text or fill_text(text, x, y, max_width=None)
for i in enumerate(lyric_count):
    canvas.fill_text(i[1], random.randint(100, 700), random.randint(100, 700))

# call canvas
canvas

Canvas(height=800, width=800)

In [26]:
# 